In [29]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import numpy as np
import pandas as pd
import unicodedata
import string
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


# In[5]:
def findFiles(path): 
    return glob.glob(path)

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


# In[6]:q


names = {}
languages = []


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# (TO DO:) CHANGE FILE PATH AS NECESSARY
for filename in findFiles(r"C:\Users\aradh\Desktop\Fall 22\TSA\Project 3.1\data\data\names\*.txt"):
    category = os.path.splitext(os.path.basename(filename))[0]
    languages.append(category)
    lines = readLines(filename)
    names[category] = lines


# In[7]:

def letterToIndex(letter):
    return all_letters.find(letter)


def nameToTensor(name):
    tensor = torch.zeros(len(name), 1, n_letters)
    for li, letter in enumerate(name):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


# In[54]:


class RNN(nn.Module):
    def __init__(self, INPUT_SIZE, HIDDEN_SIZE, N_LAYERS,OUTPUT_SIZE):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size = INPUT_SIZE,
            hidden_size = HIDDEN_SIZE, # number of hidden units
            num_layers = N_LAYERS, # number of layers
            batch_first = True)
        self.out = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
            
    def forward(self, x):
        r_out, h = self.rnn(x, None) # None represents zero initial hidden state           
        out = self.out(r_out[:, -1, :])
        return out
    

# In[8]:

#list comprehension:
# list_data=[]
# for category in languages:
#     for name in names[category]:
#         list_data.append((name, category))

n_hidden = 128

allnames = [] # Create list of all names and corresponding output language
for language in list(names.keys()):
    for name in names[language]:
        allnames.append([name, language])
        

n = 1000
x = [allnames[i:i + n] for i in range(0, len(allnames), n)]
#print(x)
    
# for category in list_data:
#     for name in names[category]:
#         allnames.append([name, category])
        
## (TO DO:) Determine Padding length (this is the length of the longest string) 

# maxlen = ..... # Add code here to compute the maximum length of string        
n_letters = len(all_letters)
n_categories = len(languages)

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i.item()
    return languages[category_i], category_i

           


# In[13]:


learning_rate = 0.005
rnn = RNN(n_letters, 128, 1, n_categories)
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)   # optimize all rnn parameters
loss_func = nn.CrossEntropyLoss()  
accuracies = []
for epoch in range(5):  
    for batch_in_all_names in x:
        batch_size = len(allnames)
        random.shuffle(allnames)
        maxlen = max(len(x[0]) for x in batch_in_all_names)
        b_in = torch.zeros(batch_size, maxlen, n_letters)  # (TO DO:) Initialize "b_in" to a tensor with size of input (batch size, padded_length, n_letters)
        b_out = torch.zeros(batch_size, n_categories, dtype=torch.long)  # (TO DO:) Initialize "b_out" to tensor with size (batch_size, n_categories, dtype=torch.long)       
        
        def get(charachter):
            return [char for char in charachter]
        for i in batch_in_all_names:
            j=batch_in_all_names.index(i)       
            k=get(i[0])
            for l in range(len(i[0])):
                b_in[j][l][letterToIndex(k[l])]=1
            m=i[1]
            l=languages.index(m)
            b_out[j][l]=1   
        max_b_out=torch.max(b_out,1)[1]
        output = rnn(b_in)                               # rnn output
        #(TO DO:)
        loss = loss_func(output, max_b_out)   # (TO DO:) Fill "...." to calculate the cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        # Print accuracy
        test_output = rnn(b_in)                   # 
        pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
        test_y = torch.max(b_out, 1)[1].data.numpy().squeeze()
        accuracy = sum(pred_y == test_y)/(2*batch_size)
        print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| accuracy: %.3f' % accuracy)
        accuracies.append(round(accuracy,3))

print(np.mean(accuracies))
        
        





Epoch:  0 | train loss: 2.9395 | accuracy: 0.500
Epoch:  0 | train loss: 2.4891 | accuracy: 0.500
Epoch:  0 | train loss: 1.4418 | accuracy: 0.476
Epoch:  0 | train loss: 0.4789 | accuracy: 0.475
Epoch:  0 | train loss: 0.3472 | accuracy: 0.475
Epoch:  0 | train loss: 0.3316 | accuracy: 0.475
Epoch:  0 | train loss: 0.3393 | accuracy: 0.475
Epoch:  0 | train loss: 0.4065 | accuracy: 0.476
Epoch:  0 | train loss: 0.4365 | accuracy: 0.475
Epoch:  0 | train loss: 0.4874 | accuracy: 0.475
Epoch:  0 | train loss: 0.4755 | accuracy: 0.475
Epoch:  0 | train loss: 0.4649 | accuracy: 0.475
Epoch:  0 | train loss: 0.4366 | accuracy: 0.475
Epoch:  0 | train loss: 0.4097 | accuracy: 0.475
Epoch:  0 | train loss: 0.3782 | accuracy: 0.475
Epoch:  0 | train loss: 0.3429 | accuracy: 0.475
Epoch:  0 | train loss: 0.3077 | accuracy: 0.475
Epoch:  0 | train loss: 0.2701 | accuracy: 0.475
Epoch:  0 | train loss: 0.2366 | accuracy: 0.476
Epoch:  0 | train loss: 0.3417 | accuracy: 0.475
Epoch:  0 | train lo

KeyboardInterrupt: 

In [20]:
batch_size

74

In [30]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import numpy as np
import pandas as pd
import unicodedata
import string
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


# In[5]:
def findFiles(path): 
    return glob.glob(path)

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


# In[6]:q


names = {}
languages = []


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# (TO DO:) CHANGE FILE PATH AS NECESSARY
for filename in findFiles(r"C:\Users\aradh\Desktop\Fall 22\TSA\Project 3.1\data\data\names\*.txt"):
    category = os.path.splitext(os.path.basename(filename))[0]
    languages.append(category)
    lines = readLines(filename)
    names[category] = lines


# In[7]:

def letterToIndex(letter):
    return all_letters.find(letter)


def nameToTensor(name):
    tensor = torch.zeros(len(name), 1, n_letters)
    for li, letter in enumerate(name):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


# In[54]:


class RNN(nn.Module):
    def __init__(self, INPUT_SIZE, HIDDEN_SIZE, N_LAYERS,OUTPUT_SIZE):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size = INPUT_SIZE,
            hidden_size = HIDDEN_SIZE, # number of hidden units
            num_layers = N_LAYERS, # number of layers
            batch_first = True)
        self.out = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
            
    def forward(self, x):
        r_out, h = self.rnn(x, None) # None represents zero initial hidden state           
        out = self.out(r_out[:, -1, :])
        return out
    

# In[8]:

#list comprehension:
# list_data=[]
# for category in languages:
#     for name in names[category]:
#         list_data.append((name, category))

n_hidden = 128

allnames = [] # Create list of all names and corresponding output language
for language in list(names.keys()):
    for name in names[language]:
        allnames.append([name, language])
        

#def randomChoice(l, size):
 #   h=[]
  #  random.shuffle(l)
   # for i in range(size):
    #    h.append(l[random.randint(0, len(l) - 1)])
        
   # return h

# allnames1000=randomChoice(allnames, 1000)

n = 2000
x = [allnames[i:i + n] for i in range(0, len(allnames), n)]
#print(x)
    
# for category in list_data:
#     for name in names[category]:
#         allnames.append([name, category])
        
## (TO DO:) Determine Padding length (this is the length of the longest string) 

# maxlen = ..... # Add code here to compute the maximum length of string        
n_letters = len(all_letters)
n_categories = len(languages)

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i.item()
    return languages[category_i], category_i

           


# In[13]:


learning_rate = 0.005
rnn = RNN(n_letters, 128, 1, n_categories)
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)   # optimize all rnn parameters
loss_func = nn.CrossEntropyLoss()  
accuracies = []
for epoch in range(5):  
    
    for batch_in_all_names in x:
        maxlen = max(len(x[0]) for x in batch_in_all_names)
        random.shuffle(batch_in_all_names)
        batch_size = len(batch_in_all_names)
        
        b_in = torch.zeros(batch_size, maxlen, n_letters)  # (TO DO:) Initialize "b_in" to a tensor with size of input (batch size, padded_length, n_letters)
        b_out = torch.zeros(batch_size, n_categories, dtype=torch.long)  # (TO DO:) Initialize "b_out" to tensor with size (batch_size, n_categories, dtype=torch.long)       
        def split(word):
            return [char for char in word]
        # (TO DO:) Populate "b_in" tensor
        for name in batch_in_all_names:
            i=batch_in_all_names.index(name)       
            list1=split(name[0])
            for m in range(len(name[0])):
                b_in[i][m][letterToIndex(list1[m])]=1



        # (TO DO:) Populate "b_out" tensor
        for name in batch_in_all_names:       
            i=batch_in_all_names.index(name)       
            lan=name[1]
            l=languages.index(lan)
            b_out[i][l]=1


        labels=torch.max(b_out,1)[1]
        output = rnn(b_in)                               # rnn output
        #(TO DO:)
        loss = loss_func(output, labels)   # (TO DO:) Fill "...." to calculate the cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        # Print accuracy
        test_output = rnn(b_in)                   # 
        pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
        test_y = torch.max(b_out, 1)[1].data.numpy().squeeze()
        accuracy = sum(pred_y == test_y)/batch_size
        print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| accuracy: %.3f' % accuracy)
        accuracies.append(round(accuracy,3))
print(np.mean(accuracies))
        
        





Epoch:  0 | train loss: 2.8151 | accuracy: 10.037
Epoch:  0 | train loss: 2.4356 | accuracy: 9.062
Epoch:  0 | train loss: 1.4394 | accuracy: 9.037
Epoch:  0 | train loss: 0.6521 | accuracy: 9.061
Epoch:  0 | train loss: 0.6645 | accuracy: 9.044
Epoch:  0 | train loss: 0.6180 | accuracy: 9.038
Epoch:  0 | train loss: 0.5982 | accuracy: 9.046
Epoch:  0 | train loss: 0.5649 | accuracy: 9.040
Epoch:  0 | train loss: 0.5116 | accuracy: 9.041
Epoch:  0 | train loss: 0.5360 | accuracy: 9.055
Epoch:  0 | train loss: 0.0696 | accuracy: 270.297
Epoch:  1 | train loss: 0.0594 | accuracy: 10.037
Epoch:  1 | train loss: 0.8191 | accuracy: 9.062
Epoch:  1 | train loss: 0.6978 | accuracy: 9.037
Epoch:  1 | train loss: 0.7533 | accuracy: 9.061
Epoch:  1 | train loss: 0.7378 | accuracy: 9.044
Epoch:  1 | train loss: 0.4044 | accuracy: 9.038
Epoch:  1 | train loss: 0.3570 | accuracy: 9.046
Epoch:  1 | train loss: 0.3660 | accuracy: 9.040
Epoch:  1 | train loss: 0.3703 | accuracy: 9.041
Epoch:  1 | trai

In [31]:
batch_size

74

In [32]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import numpy as np
import pandas as pd
import unicodedata
import string
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


# In[5]:
def findFiles(path): 
    return glob.glob(path)

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


# In[6]:q


names = {}
languages = []


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# (TO DO:) CHANGE FILE PATH AS NECESSARY
for filename in findFiles(r"C:\Users\aradh\Desktop\Fall 22\TSA\Project 3.1\data\data\names\*.txt"):
    category = os.path.splitext(os.path.basename(filename))[0]
    languages.append(category)
    lines = readLines(filename)
    names[category] = lines


# In[7]:

def letterToIndex(letter):
    return all_letters.find(letter)


def nameToTensor(name):
    tensor = torch.zeros(len(name), 1, n_letters)
    for li, letter in enumerate(name):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


# In[54]:


class RNN(nn.Module):
    def __init__(self, INPUT_SIZE, HIDDEN_SIZE, N_LAYERS,OUTPUT_SIZE):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size = INPUT_SIZE,
            hidden_size = HIDDEN_SIZE, # number of hidden units
            num_layers = N_LAYERS, # number of layers
            batch_first = True)
        self.out = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
            
    def forward(self, x):
        r_out, h = self.rnn(x, None) # None represents zero initial hidden state           
        out = self.out(r_out[:, -1, :])
        return out
    

# In[8]:

#list comprehension:
# list_data=[]
# for category in languages:
#     for name in names[category]:
#         list_data.append((name, category))

n_hidden = 128

allnames = [] # Create list of all names and corresponding output language
for language in list(names.keys()):
    for name in names[language]:
        allnames.append([name, language])
        

n = 2000
x = [allnames[i:i + n] for i in range(0, len(allnames), n)]
#print(x)
    
# for category in list_data:
#     for name in names[category]:
#         allnames.append([name, category])
        
## (TO DO:) Determine Padding length (this is the length of the longest string) 

# maxlen = ..... # Add code here to compute the maximum length of string        
n_letters = len(all_letters)
n_categories = len(languages)

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i.item()
    return languages[category_i], category_i

           


# In[13]:


learning_rate = 0.005
rnn = RNN(n_letters, 128, 1, n_categories)
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)   # optimize all rnn parameters
loss_func = nn.CrossEntropyLoss()  
accuracies = []
for epoch in range(5):  
    for batch_in_all_names in x:
        batch_size = len(allnames)
        random.shuffle(allnames)
        maxlen = max(len(x[0]) for x in batch_in_all_names)
        b_in = torch.zeros(batch_size, maxlen, n_letters)  # (TO DO:) Initialize "b_in" to a tensor with size of input (batch size, padded_length, n_letters)
        b_out = torch.zeros(batch_size, n_categories, dtype=torch.long)  # (TO DO:) Initialize "b_out" to tensor with size (batch_size, n_categories, dtype=torch.long)       
        
        def get(charachter):
            return [char for char in charachter]
        for i in batch_in_all_names:
            j=batch_in_all_names.index(i)       
            k=get(i[0])
            for l in range(len(i[0])):
                b_in[j][l][letterToIndex(k[l])]=1
            m=i[1]
            l=languages.index(m)
            b_out[j][l]=1   
        max_b_out=torch.max(b_out,1)[1]
        output = rnn(b_in)                               # rnn output
        #(TO DO:)
        loss = loss_func(output, max_b_out)   # (TO DO:) Fill "...." to calculate the cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        # Print accuracy
        test_output = rnn(b_in)                   # 
        pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
        test_y = torch.max(b_out, 1)[1].data.numpy().squeeze()
        accuracy = sum(pred_y == test_y)/(2*batch_size)
        print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| accuracy: %.3f' % accuracy)
        accuracies.append(round(accuracy,3))

print(np.mean(accuracies))
        
        





Epoch:  0 | train loss: 2.8179 | accuracy: 0.500
Epoch:  0 | train loss: 2.3750 | accuracy: 0.451
Epoch:  0 | train loss: 1.3329 | accuracy: 0.450
Epoch:  0 | train loss: 0.6214 | accuracy: 0.451
Epoch:  0 | train loss: 0.6021 | accuracy: 0.451
Epoch:  0 | train loss: 0.7154 | accuracy: 0.450
Epoch:  0 | train loss: 0.6970 | accuracy: 0.451
Epoch:  0 | train loss: 0.6546 | accuracy: 0.450
Epoch:  0 | train loss: 0.5886 | accuracy: 0.450
Epoch:  0 | train loss: 0.5733 | accuracy: 0.451
Epoch:  0 | train loss: 0.0655 | accuracy: 0.498
Epoch:  1 | train loss: 0.0523 | accuracy: 0.500
Epoch:  1 | train loss: 0.7298 | accuracy: 0.451
Epoch:  1 | train loss: 0.5546 | accuracy: 0.450
Epoch:  1 | train loss: 0.7008 | accuracy: 0.451
Epoch:  1 | train loss: 0.7181 | accuracy: 0.451
Epoch:  1 | train loss: 0.4232 | accuracy: 0.450
Epoch:  1 | train loss: 0.3756 | accuracy: 0.451
Epoch:  1 | train loss: 0.3815 | accuracy: 0.450
Epoch:  1 | train loss: 0.3799 | accuracy: 0.450
Epoch:  1 | train lo

In [33]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import numpy as np
import pandas as pd
import unicodedata
import string
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


# In[5]:
def findFiles(path): 
    return glob.glob(path)

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


# In[6]:q


names = {}
languages = []


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# (TO DO:) CHANGE FILE PATH AS NECESSARY
for filename in findFiles(r"C:\Users\aradh\Desktop\Fall 22\TSA\Project 3.1\data\data\names\*.txt"):
    category = os.path.splitext(os.path.basename(filename))[0]
    languages.append(category)
    lines = readLines(filename)
    names[category] = lines


# In[7]:

def letterToIndex(letter):
    return all_letters.find(letter)


def nameToTensor(name):
    tensor = torch.zeros(len(name), 1, n_letters)
    for li, letter in enumerate(name):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


# In[54]:


class RNN(nn.Module):
    def __init__(self, INPUT_SIZE, HIDDEN_SIZE, N_LAYERS,OUTPUT_SIZE):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size = INPUT_SIZE,
            hidden_size = HIDDEN_SIZE, # number of hidden units
            num_layers = N_LAYERS, # number of layers
            batch_first = True)
        self.out = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
            
    def forward(self, x):
        r_out, h = self.rnn(x, None) # None represents zero initial hidden state           
        out = self.out(r_out[:, -1, :])
        return out
    

# In[8]:

#list comprehension:
# list_data=[]
# for category in languages:
#     for name in names[category]:
#         list_data.append((name, category))

n_hidden = 128

allnames = [] # Create list of all names and corresponding output language
for language in list(names.keys()):
    for name in names[language]:
        allnames.append([name, language])
        
random.shuffle(allnames)
n = 1000
x = [allnames[i:i + n] for i in range(0, len(allnames), n)]
#print(x)
    
# for category in list_data:
#     for name in names[category]:
#         allnames.append([name, category])
        
## (TO DO:) Determine Padding length (this is the length of the longest string) 

# maxlen = ..... # Add code here to compute the maximum length of string        
n_letters = len(all_letters)
n_categories = len(languages)

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i.item()
    return languages[category_i], category_i

           


# In[13]:


learning_rate = 0.005
rnn = RNN(n_letters, 128, 1, n_categories)
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)   # optimize all rnn parameters
loss_func = nn.CrossEntropyLoss()  
accuracies = []
for epoch in range(5):  
    for batch_in_all_names in x:
        batch_size = len(batch_in_all_names)
        
        maxlen = max(len(x[0]) for x in batch_in_all_names)
        b_in = torch.zeros(batch_size, maxlen, n_letters)  # (TO DO:) Initialize "b_in" to a tensor with size of input (batch size, padded_length, n_letters)
        b_out = torch.zeros(batch_size, n_categories, dtype=torch.long)  # (TO DO:) Initialize "b_out" to tensor with size (batch_size, n_categories, dtype=torch.long)       
        
        def get(charachter):
            return [char for char in charachter]
        for i in batch_in_all_names:
            j=batch_in_all_names.index(i)       
            k=get(i[0])
            for l in range(len(i[0])):
                b_in[j][l][letterToIndex(k[l])]=1
            m=i[1]
            l=languages.index(m)
            b_out[j][l]=1   
        max_b_out=torch.max(b_out,1)[1]
        output = rnn(b_in)                               # rnn output
        #(TO DO:)
        loss = loss_func(output, max_b_out)   # (TO DO:) Fill "...." to calculate the cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        # Print accuracy
        test_output = rnn(b_in)                   # 
        pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
        test_y = torch.max(b_out, 1)[1].data.numpy().squeeze()
        accuracy = sum(pred_y == test_y)/(batch_size)
        print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| accuracy: %.3f' % accuracy)
        accuracies.append(round(accuracy,3))

print(np.mean(accuracies))
        
        





Epoch:  0 | train loss: 2.8500 | accuracy: 0.450
Epoch:  0 | train loss: 2.6624 | accuracy: 0.479
Epoch:  0 | train loss: 2.2322 | accuracy: 0.477
Epoch:  0 | train loss: 2.0009 | accuracy: 0.470
Epoch:  0 | train loss: 1.9274 | accuracy: 0.466
Epoch:  0 | train loss: 1.9131 | accuracy: 0.193
Epoch:  0 | train loss: 1.9722 | accuracy: 0.468
Epoch:  0 | train loss: 2.0243 | accuracy: 0.457
Epoch:  0 | train loss: 1.8797 | accuracy: 0.476
Epoch:  0 | train loss: 1.8682 | accuracy: 0.454
Epoch:  0 | train loss: 1.9166 | accuracy: 0.462
Epoch:  0 | train loss: 1.9533 | accuracy: 0.455
Epoch:  0 | train loss: 1.8365 | accuracy: 0.477
Epoch:  0 | train loss: 1.8276 | accuracy: 0.483
Epoch:  0 | train loss: 1.8732 | accuracy: 0.465
Epoch:  0 | train loss: 1.8487 | accuracy: 0.476
Epoch:  0 | train loss: 1.8156 | accuracy: 0.474
Epoch:  0 | train loss: 1.9387 | accuracy: 0.460
Epoch:  0 | train loss: 1.8831 | accuracy: 0.480
Epoch:  0 | train loss: 1.7571 | accuracy: 0.494
Epoch:  0 | train lo